# Este cuaderno busca la mejor red neuronal para predecir la salida de metas

In [1]:
## Liberías necesarias 
import os
import re
import ast
import shap
import socket 
import joblib
import warnings
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import cohen_kappa_score

## Configuración de rutas
if socket.gethostname()=='SRVCBECO01':
    os.chdir('D:\shared_data\Dropbox\Vías clinicas diabetes')
elif socket.gethostname() == 'HPJP2': 
    os.chdir(r'C:\Users\juanm\Dropbox\JP_files\UR\Vías clinicas diabetes')
elif socket.gethostname() == 'CNF106054': 
    os.chdir(r'C:\Users\paul.rodriguez\Dropbox\Salud Colombia\Diabetes Sanitas\Vías clinicas diabetes')
elif socket.gethostname() == 'CNF77701': 
    os.chdir(r'C:\Users\juanpablo.martinez\Dropbox\Vías clinicas diabetes')

    
raw_path = 'Datos org\\csv_rutas\\'
created_path = "Datos creados\\"
temp_path = 'temporales\\'
dbs_path = "Datos creados\\ml_databases\\4_cat\\"


print('Directorio actual: '+ os.getcwd())

C:\Users\juanm\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Directorio actual: C:\Users\juanm\Dropbox\JP_files\UR\Vías clinicas diabetes


### Funciones 

#### Preparación de las bases de entrenamiento y validación

In [2]:
def data_prepare(df_path, vars_to_drop):
    ## Cargando la base de datos
    data_mat = pd.read_csv(df_path)
    
    ## Eliminando variables innecesarias
    data_mat.drop(vars_to_drop, axis = 1, inplace = True)
    
    return(data_mat.reset_index(drop = True))

#### Identifcando variables categóricas y numéricas en la base de datos

In [3]:
def cat_num_identify(ml_data, rename_df):
    categorical_vars_master = ['femenino', 'ERC_high', 'ERC1', 'ERC2', 'adhiere_guia', 'no_adhiere', 'adhiere_colesterol', 'analgesicos',
                                   'antiacidos', 'antihipertensivos', 'hipoglicemiantes', 'hipolipemiantes', 'nutrition_tag_max', 
                                   'exercise_tag_max', 'alcohol_tag_max', 'tobaco_tag_max', 'creatinina_missing']

    categorical_vars = list(set(ml_data.columns).intersection(categorical_vars_master))

    numerical_vars = set(ml_data.columns).difference(set(categorical_vars+['KeyAnonimo', 'year', 'base_label', 'comorbilidades', 'fuera_metas', 'tgt_label']))
    numerical_vars = list(numerical_vars)

    types_df = pd.DataFrame({'old_name' : categorical_vars+numerical_vars,
                             'type' : ['categorical']*len(categorical_vars)+['numerical']*len(numerical_vars)})

    types_df = types_df.merge(rename_df, on = 'old_name', how = 'inner')

    new_cat_vars = list(types_df.loc[types_df['type'] == 'categorical', 'new_name'].values)
    new_num_vars = list(types_df.loc[types_df['type'] == 'numerical', 'new_name'].values)
    
    return new_cat_vars, new_num_vars

#### Partición entre variables exógenas y endógenas

In [4]:
def endog_exog_select(data_mat, dep_var):
    X = data_mat.drop(dep_var, axis = 1)
    Y = data_mat.loc[:, dep_var]
    return(X, Y)

#### Wrapper de la preparación de los datos

In [5]:
def data_prepare_wrapper():
    ## Construyendo los paths de entrenamiento y validación
    train_path = dbs_path+"train\\train_db_{}_years_base_{}.csv".format(tgt_year, base_label)
    vali_path = dbs_path+"vali\\vali_db_{}_years_base_{}.csv".format(tgt_year, base_label)

    ## Listado de variables a eliminar 
    vars_to_drop = ['KeyAnonimo', 'year', 'base_label', 'tgt_label']
    if dep_var == "comorbilidades":
        vars_to_drop = vars_to_drop+['fuera_metas']
    elif dep_var == "fuera_metas":
        vars_to_drop = vars_to_drop+['comorbilidades']

    ## Diccionario para renombrar variables
    rename_dict ={'femenino' : 'Female', 'edad' : 'Age (Years)', 'peso': 'Weight (Kg)', 'talla' : 'Height (m)', 
                  'imc' : "BMI (kg/m*m)", 'Colesterol_LDL' : 'LDL Chol. (mg/dL)', 'TFG' : "eGFR (mg/g)", 
                  "ta_diastolica" : "Diast. B.P. (mmHg)", 'ta_sistolica' : 'Sist. B.P. (mmHg)', 'adhiere_guia' : "Hba1c guide Adh.", 
                  "no_adhiere" : "Pharma. Adh.", "analgesicos" : "Analgesics", "antiacidos" : "Antacdis",
                  "antihipertensivos" : "Antihypertensive", "hipoglicemiantes" : "Hypoglecimic agents", 
                  "hipolipemiantes" : "Lipid-lowering agents", "nutrition_tag_max" : "Nutrition recomm.", 
                  "exercise_tag_max" : "Physical act. recomm.", "alcohol_tag_max" : "Alcohol recomm.",
                  "tobaco_tag_max" : "Tobacco recomm.", "creatinina" : "Creatinine (mg/dL)", 
                  "adhiere_colesterol" : "Chol. Adh."} 

    #'imc': r"BMI ($\displaystyle\frac{kg}{m^2}$)"}

    rename_df = pd.DataFrame(rename_dict, index = rename_dict.keys())
    rename_df = pd.DataFrame({'old_name' : rename_df.index, 'new_name' : np.diag(rename_df)})
    
    ## Preparando los datos
    train = data_prepare(df_path = train_path, vars_to_drop = vars_to_drop)
    vali = data_prepare(df_path = vali_path, vars_to_drop = vars_to_drop)

    ## Identificando variables categóricas y numéricas presentes
    cat_vars, num_vars = cat_num_identify(ml_data = train, rename_df = rename_df)

    ## Renombrando las variables para el gráfico
    train.rename(rename_dict, axis = 1, inplace = True)
    vali.rename(rename_dict, axis = 1, inplace = True)

    ## Segmentación entre variables endógenas y exógenas 
    X_train, Y_train = endog_exog_select(data_mat = train,
                                             dep_var = 'fuera_metas')
                                             #dep_var = 'comorbilidades')
    X_vali, Y_vali = endog_exog_select(data_mat = vali, 
                                       dep_var = 'fuera_metas')
                                    #   dep_var = 'comorbilidades')
        
    return X_train, Y_train, X_vali, Y_vali

#### Incialización del modelo

In [6]:
## Función para instanciar el modelo 
def model_init(reg_param, 
               input_shape, 
               output_shape):
    model = tf.keras.Sequential([tf.keras.layers.Dense(256,
                                             activation="relu", 
                                             input_shape=[input_shape]),
                          tf.keras.layers.Dense(256, 
                                             activation="relu"),
                          tf.keras.layers.Dropout(reg_param),
                          tf.keras.layers.Dense(256,
                                             activation="relu"),
                          tf.keras.layers.Dropout(reg_param),
                         tf.keras.layers.Dense(output_shape,
                                             activation="sigmoid")
                         ])
    return model

#### Compilación y ejecución del entrenamiento

In [7]:
def model_train_exe(reg_param, 
                    batch_size,
                    epochs,
                    lr_init, 
                    weighted):
    # Incializando el modelo 
    model = model_init(reg_param = reg_param, 
                       input_shape = len(X_train.columns),
                       output_shape = len(pd.DataFrame(Y_train).columns))

    ## Compilación del modelo 
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr_init),
                  loss = "binary_crossentropy")
    
    ## Si el modelo es con pesos, computarlos y añadirlos 
    if weighted:
        ## Construcción de los pesos 
        counts = np.bincount(Y_train)
        class_weights = {0: 1.0 / counts[0], 1: 1.0 / counts[1]}
        
        ## Entrenamiento del modelo 
        model.fit(X_train,
                  Y_train,
                  validation_data = (X_vali,
                                     Y_vali),
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = 0, 
                  class_weight = class_weights
                 )

    else:
        ## Entrenamiento del modelo 
        model.fit(X_train,
                  Y_train,
                  validation_data = (X_vali,
                                     Y_vali),
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = 0, 
                #  class_weight = class_weights
                 )
            
    return model

#### Cómputo del corte óptimo

In [8]:
def optimal_cutoff_finder(predicted_probabilities, delta, endog_vars):
    f1_res = []

    for pr_limit in np.arange(0, 1, delta):
        ## Asignando la etiqueta en función del corte óptimo
        predicted_labels = np.where(predicted_probabilities>pr_limit, 1, 0)

        ## F1-score
        f1_res.append(f1_score(endog_vars, predicted_labels, average = 'weighted'))

    f1_compiled = pd.DataFrame({'f1_score' : f1_res})
    f1_compiled.index = np.arange(0, 1, delta)

    return f1_compiled.idxmax()['f1_score']

#### Compilación y ejecución de la evaluación

In [9]:
def model_evaluator(predicted_probabilities, samp_type, endog_vars, pr_limit):

    # Prediciendo etiquetas 
    predicted_labels = np.where(predicted_probabilities>pr_limit, 1, 0)

    # Computando métricas 
    accuracy = accuracy_score(endog_vars, predicted_labels)
    auc = round(roc_auc_score(y_true = endog_vars, y_score = predicted_probabilities), 4)
    recall = round(recall_score(y_true = endog_vars, y_pred = predicted_labels), 4)
    prec = round(precision_score(y_true = endog_vars, y_pred = predicted_labels), 4)
    f1 = f1_score(endog_vars, predicted_labels, average = 'weighted')
    scores = [tgt_year, base_label, dep_var, samp_type,
              weighted, epochs, reg_param, batch_size, lr_init,
              pr_limit, accuracy, auc, recall, prec, f1]

    # Generando la matriz de resultados 
    final_info = pd.DataFrame()
    final_info.loc[0, ['tgt_year', 'base_label', 'dep_var', 'samp_type', 
                       "weighted", "epochs", "reg_param", "batch_size", "lr",
                       'cutoff', 'accuracy', 'auc', 'recall', 'prec', 'f1']] = scores
    
    return final_info

def all_res_wrapper():
    ## Computando métricas con ambos cortes para entrenamiento y validación
    train_naive = model_evaluator(predicted_probabilities = train_probs, 
                    samp_type = 'training',
                    endog_vars = Y_train,
                    pr_limit = 0.5)

    train_optimal = model_evaluator(predicted_probabilities = train_probs, 
                                    samp_type = 'training',
                                    endog_vars = Y_train,
                                    pr_limit = cutoff)

    vali_naive = model_evaluator(predicted_probabilities = vali_probs, 
                                    samp_type = 'validation',
                                    endog_vars = Y_vali,
                                    pr_limit = 0.5)

    vali_optimal = model_evaluator(predicted_probabilities = vali_probs, 
                                    samp_type = 'validation',
                                    endog_vars = Y_vali,
                                    pr_limit = cutoff)

    return pd.concat([train_naive, train_optimal, vali_naive, vali_optimal], axis = 0, ignore_index = True)

## Entrenamiento para varios estadios base y horizontes temporales

In [10]:
### Parámetros básicos
## Horizonte temporal, estadío base y variable a predecir
for tgt_year in [1, 2]:
    for base_label in [1, 3]:
        dep_var = 'fuera_metas'

        ### Ejecución de los modelos
        ## Lista vacía para llenar en cada iteración 
        all_results = []

        ## Organización de los datos 
        X_train, Y_train, X_vali, Y_vali = data_prepare_wrapper()

        ## Hiperparametros red neuronal    
        for weighted in [False, True]:
            for epochs in [20, 50]:
                for reg_param in [0.2, 0.3, 0.5]:
                    for batch_size in [50, 100, 150]:
                        for lr_init in [0.0001, 0.0005, 0.001]:

                            ## Entrenamiento del modelo 
                            trained_model = model_train_exe(reg_param = reg_param, 
                                                            batch_size = batch_size, 
                                                            epochs = epochs, 
                                                            lr_init = lr_init, 
                                                            weighted = weighted)

                            ## Predicciones del modelo 
                            train_probs = trained_model.predict(X_train, verbose = 0)
                            vali_probs = trained_model.predict(X_vali, verbose = 0)

                            ## Hallando el corte óptimo
                            cutoff = optimal_cutoff_finder(predicted_probabilities = vali_probs, delta = 0.001, endog_vars = Y_vali)

                            ## Generando los resultados finales en comparación al output de pycaret
                            all_results.append(all_res_wrapper())

                            pd.concat(all_results, 
                                      axis = 0,
                                      ignore_index = True).to_csv(temp_path+'nn_res_{}_{}_years_base_{}.csv'.format(dep_var, 
                                                                                                                    tgt_year,
                                                                                                                    base_label), 
                                                                 index = False)

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zer

### Escogiendo el mejor modelo y recreándolo

In [28]:
### Parámetros básicos
## Horizonte temporal, estadío base y variable a predecir
for tgt_year in [1, 2]:
    for base_label in [1, 3]:
        dep_var = 'fuera_metas'

        ### Ejecución de los modelos
        ## Lista vacía para llenar en cada iteración 
        all_results = []

        ## Organización de los datos 
        X_train, Y_train, X_vali, Y_vali = data_prepare_wrapper()


        all_results = pd.read_csv(temp_path+'nn_res_{}_{}_years_base_{}.csv'.format(dep_var, 
                                                                                    tgt_year,
                                                                                    base_label))
        
        # Keep metrics evaluated at naive cutoff
        all_results = all_results[all_results['cutoff'] == 0.5].reset_index(drop = True)

        best_model = all_results.loc[(all_results['samp_type'] == "validation"), 'f1'].idxmax()
        best_model = all_results.loc[best_model, :]

        ## Re-entrenamiento del modelo 
        trained_model = model_train_exe(reg_param = best_model["reg_param"], 
                                        batch_size = best_model["batch_size"].astype(int), 
                                        epochs = best_model["epochs"].astype(int), 
                                        lr_init = best_model["lr"], 
                                        weighted = best_model['weighted'])

        ## Exportando el modelo 
        saving_path = created_path+"ml_models\\final\\{}_{}_years_base_{}".format(dep_var, tgt_year, base_label)
        trained_model.save(saving_path+'.keras')
        #joblib.dump(trained_model, saving_path+'.pkl')
        
        ## Exportando las probabilidades y etiquetas del modelo
        pred_res = pd.concat([pd.DataFrame({"sample" : np.repeat("training", len(X_train)), 
                                              "probabilities" : trained_model.predict(X_train, verbose = 0)[:, 0], 
                                              "labels" : Y_train}),
                                pd.DataFrame({"sample" : np.repeat("validation", len(X_vali)), 
                                              "probabilities" : trained_model.predict(X_vali, verbose = 0)[:, 0], 
                                              "labels" : Y_vali})], 
                                axis = 0)
        pred_res['dep_var'] = dep_var
        pred_res['base_label'] = base_label
        pred_res['tgt_year'] = tgt_year
        pred_res['model_type'] = "NN"

        pred_res.to_csv(temp_path+"ryr_results\\baseline_replicate\\predicted_probabilities\\{}_{}_years_base_{}.csv".format(dep_var, tgt_year, base_label), 
                        sep = ';', 
                        index = False)

tgt_year              1.0
base_label            1.0
dep_var       fuera_metas
samp_type      validation
weighted            False
epochs               20.0
reg_param             0.3
batch_size           50.0
lr                 0.0005
cutoff                0.5
accuracy         0.806867
auc                0.5805
recall             0.0968
prec               0.3429
f1               0.759751
Name: 21, dtype: object
tgt_year              1.0
base_label            3.0
dep_var       fuera_metas
samp_type      validation
weighted            False
epochs               50.0
reg_param             0.3
batch_size          100.0
lr                 0.0001
cutoff                0.5
accuracy         0.803642
auc                0.5679
recall             0.0527
prec               0.3472
f1               0.740116
Name: 79, dtype: object
tgt_year              2.0
base_label            1.0
dep_var       fuera_metas
samp_type      validation
weighted             True
epochs               20.0
reg_param       